## Import dependencies

In [2]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install transformers requests beautifulsoup4 pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

C:\Users\ASUS\anaconda3\envs\nlp_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\ASUS\anaconda3\envs\nlp_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Instantiate Model

In [2]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

## Encode and Calculate Sentiment

In [3]:
tokens = tokenizer.encode('It was okay', return_tensors='pt')

In [4]:
result = model(tokens)

In [5]:
result.logits

tensor([[-1.6243,  0.8367,  2.7918,  0.3680, -2.0830]],
       grad_fn=<AddmmBackward0>)

In [6]:
int(torch.argmax(result.logits))+1

3

## Collect Reviews

In [64]:
r = requests.get('https://www.yelp.com/biz/shabu-club-burlingame-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [74]:
reviews

["We had a great dinner at Shabu Club in Burlingame the other night. Everything from the moment we walked in was smooth and welcoming. We had a reservation and were seated right away. The staff was genuinely friendly and attentive without being overbearing.We went with the set menus instead of the all-you-can-eat, and I'm glad we did. The tonkatsu broth was delicious--really well balanced and comforting. All the veggies were super fresh, and the meat quality stood out--tender, nicely marbled, and flavorful.It's one of those places that feels like a true neighborhood spot. No fuss, just good food, great service, and a warm atmosphere. Definitely looking forward to coming back soon.",
 "First time here! This place was huge and had a lot of seating. We wanted Japanese shabu shabu, but left feeling like we ate more of a Chinese/Korean hot pot. Service was decent and the server was nice to give us different sauces since they only provided a complimentary ciltantro sauce. We ordered the gyoz

## load reviews into DataFrame and Score

In [66]:
import pandas as pd
import numpy as np

In [67]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [68]:
df.head()

,review
0,We had a great dinner at Shabu Club in Burling...
1,First time here! This place was huge and had a...
2,"The meat was just okay--nothing terrible, but ..."
3,Came here a week ago and was impressed with th...
4,Very underwhelming. Below average for Bay Area...


In [69]:
df['review'].iloc[0]

"We had a great dinner at Shabu Club in Burlingame the other night. Everything from the moment we walked in was smooth and welcoming. We had a reservation and were seated right away. The staff was genuinely friendly and attentive without being overbearing.We went with the set menus instead of the all-you-can-eat, and I'm glad we did. The tonkatsu broth was delicious--really well balanced and comforting. All the veggies were super fresh, and the meat quality stood out--tender, nicely marbled, and flavorful.It's one of those places that feels like a true neighborhood spot. No fuss, just good food, great service, and a warm atmosphere. Definitely looking forward to coming back soon."

In [70]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [71]:
sentiment_score(df['review'].iloc[0])

5

In [72]:
df['Sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [73]:
df

,review,Sentiment
0,We had a great dinner at Shabu Club in Burling...,5
1,First time here! This place was huge and had a...,4
2,"The meat was just okay--nothing terrible, but ...",3
3,Came here a week ago and was impressed with th...,4
4,Very underwhelming. Below average for Bay Area...,2
5,Decided to try Shabu Club on a Friday night wi...,2
6,"Hi Marie,Thanks for coming in and giving us a ...",4
7,"Maybe this location is still new, but it needs...",3
8,"Hi Deanna, Thanks for visiting our new locatio...",4
9,New location of Shabu Club opened up in Burlin...,4
